In [ ]:
#Labraries
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, EfficientNetV2S
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
#Connecting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Path to images
base_dir = '/content/drive/My Drive/Cancer/AugImages/'

def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

#Sample loading of images from a folder
class_names = ['Neg250', 'AugHSIL', 'AugLSIL', 'AugSCC']
dataset = {}

for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    img_paths = [os.path.join(class_dir, img_name) for img_name in os.listdir(class_dir)]
    dataset[class_name] = [(img_path, load_and_preprocess_image(img_path)) for img_path in img_paths]
print(f"Loaded {len(dataset['Neg250'])} images for class 'Neg250'")

Loaded 250 images for class 'Neg250'


In [ ]:
#Loading EfficientNetV2-S
base_model = EfficientNetV2S(weights='imagenet', include_top=False)

#Freezing all layers
for layer in base_model.layers:
    layer.trainable = False

#Adding new layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(class_names), activation='softmax')(x)  #Number of classes

#Creating a new model
model = Model(inputs=base_model.input, outputs=predictions)

#Model setup
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Модель для дообучения создана


In [ ]:
#Data splitting
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 800 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Данные для обучения и валидации подготовлены


In [ ]:
#Training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
25/25 [==============================] - 278s 10s/step - loss: 1.2201 - accuracy: 0.4050 - val_loss: 0.9207 - val_accuracy: 0.6458
Epoch 2/10
25/25 [==============================] - 292s 12s/step - loss: 1.1134 - accuracy: 0.4925 - val_loss: 0.8980 - val_accuracy: 0.6042
Epoch 3/10
25/25 [==============================] - 280s 11s/step - loss: 1.1053 - accuracy: 0.5312 - val_loss: 0.8436 - val_accuracy: 0.7344
Epoch 4/10
25/25 [==============================] - 255s 10s/step - loss: 1.0601 - accuracy: 0.5387 - val_loss: 0.9484 - val_accuracy: 0.5938
Epoch 5/10
25/25 [==============================] - 253s 10s/step - loss: 1.0323 - accuracy: 0.5550 - val_loss: 0.7955 - val_accuracy: 0.7500
Epoch 6/10
25/25 [==============================] - 286s 11s/step - loss: 1.0191 - accuracy: 0.5587 - val_loss: 0.7917 - val_accuracy: 0.7240
Epoch 7/10
25/25 [==============================] - 283s 11s/step - loss: 0.9871 - accuracy: 0.5825 - val_loss: 0.7764 - val_accuracy: 0.6250
Epoch 

In [ ]:
#Unfreezing the top layers of the model
for layer in base_model.layers[-10:]:
    layer.trainable = True

#Compiling the model with a reduced learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#Fine-tuning
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
25/25 [==============================] - 291s 11s/step - loss: 1.0380 - accuracy: 0.5625 - val_loss: 0.7807 - val_accuracy: 0.7135
Epoch 2/10
25/25 [==============================] - 255s 10s/step - loss: 0.9430 - accuracy: 0.6062 - val_loss: 0.7994 - val_accuracy: 0.7292
Epoch 3/10
25/25 [==============================] - 259s 10s/step - loss: 0.8930 - accuracy: 0.6413 - val_loss: 0.7390 - val_accuracy: 0.7135
Epoch 4/10
25/25 [==============================] - 286s 11s/step - loss: 0.9092 - accuracy: 0.6338 - val_loss: 0.7675 - val_accuracy: 0.6823
Epoch 5/10
25/25 [==============================] - 292s 12s/step - loss: 0.9041 - accuracy: 0.6425 - val_loss: 0.7485 - val_accuracy: 0.6719
Epoch 6/10
25/25 [==============================] - 285s 11s/step - loss: 0.8566 - accuracy: 0.6525 - val_loss: 0.7335 - val_accuracy: 0.7240
Epoch 7/10
25/25 [==============================] - 295s 12s/step - loss: 0.8620 - accuracy: 0.6600 - val_loss: 0.6916 - val_accuracy: 0.7865
Epoch 

In [ ]:
#Feature extraction model
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
x = GlobalAveragePooling2D()(feature_extractor.output)
feature_model = Model(inputs=feature_extractor.input, outputs=x)

Модель для извлечения признаков создана


In [ ]:
#Feature extraction function
def extract_features(img_array):
    return feature_model.predict(img_array).flatten()

#Feature extraction for all images and saving them to CSV files
for class_name, img_list in dataset.items():
    features = []
    filenames = []
    for img_path, img_array in img_list:
        feature_vector = extract_features(img_array)
        features.append(feature_vector)
        filenames.append(os.path.basename(img_path))

    #Creating a DataFrame with filenames and features
    df = pd.DataFrame(features)
    df.insert(0, 'filename', filenames)

    #Saving the DataFrame to a CSV file
    df.to_csv(f'/content/drive/My Drive/extracted_features_{class_name}.csv', index=False)

#Checking
df_loaded = pd.read_csv('/content/drive/My Drive/extracted_features_Neg250.csv')
print(f"Loaded features shape from CSV: {df_loaded.shape}")

1/1 [==============================] - 0s 195ms/step
Loaded features shape from CSV: (250, 1281)
